<a href="https://colab.research.google.com/github/whtngus/2018_molit_3H/blob/master/RAG_with_LLama3_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [ ]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

약 6분 소요

In [ ]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Evo"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

## 4단계 - 잘 실행되는지 확인

약 1분 소요

In [ ]:
device = "cuda:0"

messages = [
    {"role": "system", "content": "당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요."},
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
model_inputs = encodeds.to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

generated_ids = model.generate(model_inputs, max_new_tokens=512, eos_token_id=terminators, do_sample=True, repetition_penalty=1.05,)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

여러분~ 은행의 기준 금리(the benchmark interest rate)에 대해 궁금하시죠~

기준 금리란, 중앙은행이 연방공개시장(Open Market Operations)에서 자산 또는 부채의 수급 조작을 통해 경기와 경제 성장을 조절하는 정책인 통화정책의 기본 수단 중 하나입니다. 이를 통해, 경제 성장률, 실물경제의 비용, 저축 및 투자의 흐름 등을 조절할 목적으로 설정된 금리입니다.

자본금리는, 다음의 기능을 합니다.

1. 경제성장: 기준 금리가 낮아지면 기업의 자금조달 비용이 떨어져 경제 성장에 한층 이혜한 기를 만들며, 소비자들은 신용을 빌릴 수 있게 해 경제 활동을 촉진합니다.
2. 저축 및 투자: 기준 금리가 높아지면 자금 조달이 어려워 서서히 지갑을 덜 쓰면서 저축과 투자를 늘려 경제 안정화를 기할 수 있습니다.

기준 금리가 왜 중요한가요?

1. 자산과 채권 가격의 변동: 기준 금리의 조정은 자산과 채권 가격의 변동을 가져오는데, 이는 주택 구입 등에 큰 영향을 미칠 수 있습니다.
2. 가계 경비: 고금리 시기에는 가계에 큰 경보를 가져오지만, 저금리 시기에는 가계에 한층 더 유리한 조건을 제공합니다.
3. 기업의 투자 결정: 기준 금리 증가에 따라 기업의 투자 결정에 막대한 영향을 미치고, 이로 인한 생산성 변화는 경제 전반적인 영향을 가져올 수 있습니다.
4. 국제 금융: 기준 금리의 조정은 국제금융 시장에서 환율이나 채권금리를 좌우하는데, 이는 다른 나라들의 경제 대응에도 영향을 미칩니다.

이러한 기준 금리 정보가 도움

이번에는 RAG로 실험해보자.

In [ ]:
search_result = '''쏘카(대표 이재웅)가 해군(해군참모총장 심승섭 대장)과 공유경제 활성화 및 업무 효율 향상을 위한 업무 협약을 체결했다. 해군은 국군 중 최초로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 도입한다. 쏘카와 해군은 지난 5일 서울 해군 재경근무지원대대 회의실에서 김남희 쏘카 신규사업본부장과 조정권 해군본부 군수차장과 김은석 해군본부 수송과장 등 주요 관계자가 참석한 가운데 상호협력과 발전을 위한 업무 협약식을 체결했다. 양 기관은 △해군본부 임직원의 업무 이동 효율성 향상 △공유 차량을 활용한 해군 본부 및 부대 주차난 해소 △공유 차량 이용 활성화 및 확대를 위해 적극 협력키로 했다. 쏘카는 해군 장병과 군무원을 대상으로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 제공한다. 해군 장병들과 군무원은 업무 이동 시 전국 쏘카존에 있는 1만 2천여 대의 차량을 이용할 수 있다. 특히, 출장 시에는 전국 74개 시군의 KTX, 기차역, 버스터미널, 공항 등 대중교통과 교통 편의시설 거점이 연결된 260여 개 쏘카존을 통해 효율적인 이동이 가능해진다. 쏘카와 해군은 우선 올해까지 해군본부를 대상으로 ‘쏘카 비즈니스’ 시범 적용을 거친 후 내년부터는 해군 전 부대로 확대할 계획이다. 그전까지 일반 사병들에게는 별도로 월별 할인 혜택과 특전을 제공, 휴가와 외출 시에도 합리적인 가격으로 쏘카를 이용할 수 있도록 지원한다. 조정권 해군본부 군수차장은 “해군은 장병들의 업무 이동 편의성을 향상시키는 동시에 사기진작과 복리 증진을 위해 차량 공유 서비스 기업과 업무협약을 체결했다”며 “전문기관, 업체와의 협력을 통해 새로운 기술을 해군 수송업무에 도입해 해군이 그려나가는 ‘스마트 해군’ 건설에 한 걸음 더 다가갈 수 있도록 노력하겠다”고 말했다. 김남희 쏘카 신규사업본부장은 “일반 기업체 외에도 군이나 지자체, 공공기관 등에서도 법인용 차량 공유 서비스에 대한 수요가 꾸준히 늘어나고 있다”며 “업무 이용 패턴과 특성에 맞는 서비스 인프라와 라인업을 지속해서 확대해 나갈 것”이라고 말했다.'''

user_prompt = '검색 결과:\n' + search_result + '\n\n질문: 해군이 쏘카와 도입하는 서비스는?'

messages = [
    {"role": "system", "content": "당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요."},
    {"role": "user", "content": user_prompt}
]

encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
model_inputs = encodeds.to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

generated_ids = model.generate(model_inputs, max_new_tokens=512, eos_token_id=terminators, do_sample=True, repetition_penalty=1.05,)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

검색 결과:
쏘카(대표 이재웅)가 해군(해군참모총장 심승섭 대장)과 공유경제 활성화 및 업무 효율 향상을 위한 업무 협약을 체결했다. 해군은 국군 중 최초로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 도입한다. 쏘카와 해군은 지난 5일 서울 해군 재경근무지원대대 회의실에서 김남희 쏘카 신규사업본부장과 조정권 해군본부 군수차장과 김은석 해군본부 수송과장 등 주요 관계자가 참석한 가운데 상호협력과 발전을 위한 업무 협약식을 체결했다. 양 기관은 △해군본부 임직원의 업무 이동 효율성 향상 △공유 차량을 활용한 해군 본부 및 부대 주차난 해소 △공유 차량 이용 활성화 및 확대를 위해 적극 협력키로 했다. 쏘카는 해군 장병과 군무원을 대상으로 법인용 차량 공유 서비스 ‘쏘카 비즈니스’를 제공한다. 해군 장병들과 군무원은 업무 이동 시 전국 쏘카존에 있는 1만 2천여 대의 차량을 이용할 수 있다. 특히, 출장 시에는 전국 74개 시군의 KTX, 기차역, 버스터미널, 공항 등 대중교통과 교통 편의시설 거점이 연결된 260여 개 쏘카존을 통해 효율적인 이동이 가능해진다. 쏘카와 해군은 우선 올해까지 해군본부를 대상으로 ‘쏘카 비즈니스’ 시범 적용을 거친 후 내년부터는 해군 전 부대로 확대할 계획이다. 그전까지 일반 사병들에게는 별도로 월별 할인 혜택과 특전을 제공, 휴가와 외출 시에도 합리적인 가격으로 쏘카를 이용할 수 있도록 지원한다. 조정권 해군본부 군수차장은 “해군은 장병들의 업무 이동 편의성을 향상시키는 동시에 사기진작과 복리 증진을 위해 차량 공유 서비스 기업과 업무협약을 체결했다”며 “전문기관, 업체와의 협력을 통해 새로운 

## 5단계- RAG 시스템 결합하기

In [ ]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

검색 결과: {context}

질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
loader = PyPDFLoader("/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [ ]:
rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d0f55019120>, search_kwargs={'k': 3}),
  question: RunnablePassthrough()
}
| LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n검색 결과: {context}\n질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d0f678f3160>))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
result = rag_chain.invoke("세계 인공지능 시장규모는?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print('--' * 50)
print(f"\n답변: {result['text']}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


주어진 근거: ￮세부 솔루션 분문별로는 2020 년 기준 소프트웨어 부문의 점유율이 전체시장의 78.3% 를 차지할 
정도로 압도적으로 높음
[세계 인공지능 시장규모 ]
                                                            (단위: 억 달러, 괄호는 YoY %)
구분 2020 2021 2022 2023 2024 2025CAGR
(2020-2025)
인공지능398.4 553.3 769.7 1,134.3 1,498.9 2,223.7 41.0%
　 (38.9) (39.1) (47.4) (32.1) (48.4)   
소프트웨어311.8 432.3 600.3 882.4 1,164.6 1,723.5 40.8%
　 (38.6) (38.8) (47.0) (32.0) (48.0)   
서비스55.7 78.0 109.3 162.6 216.0 323.7 42.2%
　 (40.0) (40.2) (48.9) (32.8) (49.8) / 출처: /content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf - 9 


주어진 근거: | 8 | CIS이슈리포트 2022-2 호 ▶(주요품목 ② : 인공지능 ) 정보통신 테마 내 기술분야 중 정책금융 공급규모 증가율이 가장 높은 
능동형컴퓨팅 분야의 경우, 인공지능 품목의 정책금융 공급 비중이 가장 높으며 , 이는 빅데이터 
분석기술의 발전으로 인해 인공지능의 활용처가 넓어짐에 따른 것으로 분석됨
[능동형컴퓨팅 분야 내 기술품목별 혁신성장 정책금융 공급액 추이]
(단위: 억 원)
주: 스마트물류시스템 품목은 2021 년부터 신규 품목으로 편임
▶인공지능은 인간의 학습능력과 추론·지각능력 , 자연언어 이해능력 등을 프로그램으로 구현한 기술로 , 
컴퓨터가 인간의 지능적인 행동을 모방하는 방향으로 발전하고 있음
￮인공지능은 사람의 두뇌가 복잡한 연산을 수행하는 점을 모방해 뉴런(Neuron) 을 수학적으로 모방한  
알고리즘인 퍼셉트론 (Perceptron) 을 이용하여 컴퓨터의